In [3]:
# Import necessary libraries
from PyMovieDb import IMDB
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd

import getpass

import os
import openai


In [4]:
# Define headers for HTTP request to give and impression of the site being browsed as a user than a machine
HEADERS ={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}


In [31]:
openai.api_key = getpass.getpass("Enter OpenAI API Key: ")

Enter OpenAI API Key: ········


In [5]:
# Create an instance of IMDB class from PyMovieDb library
imdb = IMDB()

In [35]:
# Ask user for movie name to search
user_input = input("Enter a movie to search: ")

Enter a movie to search: justice league


In [36]:
# Search for the movie using the IMDB class instance
# and get the result as a JSON object
# i am not looking at TV shows so have tagged it as false

res = imdb.get_by_name(user_input, tv=False)
#the result came as string so to run it, used json to query

In [37]:
# Convert the JSON object to Python dictionary using json module

res_json = json.loads(res)

# Upon inspection of movies page, all users review have a commom url string which we are adding to the basic url of movie
#to ger to movies's review page directly (Via Network page i identified the pages being acccessed)
movie_review_url = res_json["url"]+"reviews?ref_=tt_urv"

In [38]:
print(movie_review_url)

https://www.imdb.com/title/tt0974015/reviews?ref_=tt_urv


In [39]:
# Send a HTTP GET request to the movie's reviews page using requests library
# and pass the headers defined earlier
movie_review_response = requests.get(movie_review_url, headers=HEADERS)

#using html parser
movie_review_response_content = BeautifulSoup(movie_review_response.content, 'lxml')


In [40]:
# Select all review containers on the page and store them in a list
movie_review = movie_review_response_content.select(".review-container")

In [41]:
# Loop through first 5 reviews and perform sentiment analysis on each
for index in range(3):
    # Print movie name and review number
    print("\n")
    print("Movie: ", end="")
    print(user_input)
    print("Review ", end="")
    print(index+1, end="")
    print("\n\n")
    print(movie_review[index].select(".text")[0].text) # Here red .text means where . = class and the black #the .text here picks up the text of any BeautifulSoup html
    print("\n")

    # Call OpenAI's GPT-3 API to classify sentiment in the review (used prompt from OpenAI Advanced Tweet Classifier)
    response = openai.Completion.create(
    model="text-davinci-003",
    prompt="Classify the sentiment in this review:\n\n"+movie_review[index].select(".text")[0].text, 
    temperature=0,  #randomness
    max_tokens=500,#characters in response
    top_p=1.0, #controls diversity
    frequency_penalty=0.0, #decrease repetition 
    presence_penalty=0.0 #increase likelihood to talk about new topic 
    )
     # Get the sentiment classification only from the API response
    sentiment = response["choices"][0]["text"]
    print(sentiment)
    print("\n")




Movie: justice league
Review 1


After watching the Snyder cut i will never ever acknowledge this cut the josstice league , doesn't matter now this terrible corny tacky version is dead to me and surely every comic book fan you don't have to like Dc or love it you can hate it but once you watch the Snyder cut you will be like wow this is a comic book masterpiece.




Positive




Movie: justice league
Review 2


Here we are. The Justice League. DC's greatest superheroes come together to save the world...and what is the first scene you ask? Henry Cavill's CGI'd shaved face. Ohhh dear...yup...we're in for quite a ride. So with Superman dead, Batman and Wonder Woman must arrange a team of gifted individuals to stop the world destroyer Steppenwolf from, you guessed it, destroying the world. Sighhhh...I kinda just want to hug this film and repeatedly whisper "there, there...it'll all be alright.". Apprehensively, I went into this unmotivated and expecting the worst superhero film since Sui